In [34]:
import pandas as pd

ratings1_df = pd.read_csv("Data/ratings1.csv", sep=";")
ratings2_df = pd.read_csv("Data/ratings2.csv", sep=";")
movies_df = pd.read_csv("Data/movies.csv", sep=";")
dates_df = pd.read_csv("Data/dates.csv", sep=";")

In [35]:
ratings1 = ratings1_df.copy()
ratings2 = ratings2_df.copy()
movies = movies_df.copy()
dates = dates_df.copy()

Задание 5.2
Сколько уникальных фильмов представлено в таблице movies?

In [26]:
display(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


None

Задание 5.3
Сколько уникальных пользователей в таблице ratings1?

In [27]:
display(ratings1['userId'].nunique())

274

Задание 5.4
В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу dates.

In [37]:
dates['date'] = pd.to_datetime(dates['date'], dayfirst=True)
display(dates.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    100836 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 787.9 KB


None

In [56]:
dates['date'].dt.year.value_counts().index[0]

2000

Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его. 

In [58]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой:

In [59]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


In [60]:
display(ratings1.tail(1))
display(ratings2.head(1))

,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [61]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы ratings и dates по столбцам:

In [62]:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on.

In [65]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Посмотрим на метод merge() в действии. Произведём слияние наших таблиц и получим ту же таблицу, что и ранее:

In [66]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:

In [67]:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).

In [68]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

Задание 8.1
Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
У скольких фильмов не указан год их выпуска?

In [78]:
merged['year_release'] = merged['title'].apply(get_year_release).astype('Int64') # type: ignore

In [79]:
display(merged.head())

,userId,movieId,rating,date,title,genres,year_release
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [94]:
print(merged['userId'].count()-merged['year_release'].count())

18


Задание 8.2
Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
В качестве ответа запишите название этого фильма без указания года его выпуска.

In [127]:
mask = merged['year_release'] == 1999
filtered = merged[mask]
filtered_group = filtered.groupby(by=['title'])['rating'].agg(['mean'])
result = filtered_group.sort_values(by='mean', ascending=True)
print(result.head(1))

                                    mean
title                                   
Bloodsport: The Dark Kumite (1999)   0.5


Задание 8.3
Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [128]:
mask = merged['year_release'] == 2010
filtered = merged[mask]
filtered_group = filtered.groupby(by=['genres'])['rating'].agg(['mean'])
result = filtered_group.sort_values(by='mean', ascending=True)
print(result.head(1))

               mean
genres             
Action|Sci-Fi   1.0


Задание 8.4
Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.

In [130]:
filtered_group = merged.groupby(by=['userId'])['genres'].agg(['nunique'])
result = filtered_group.sort_values(by='nunique', ascending=False)
print(result.head(1))

        nunique
userId         
599         524


Задание 8.5
Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
В качестве ответа укажите идентификатор этого пользователя.
Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

In [131]:
filtered_group = merged.groupby(by=['userId'])['rating'].agg(['count', 'mean'])
result = filtered_group.sort_values(by = ['count','mean'], ascending=[True, False])
print(result.head(1))

        count  mean
userId             
53         20   5.0


Задание 8.6
Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [136]:
mask = merged['year_release'] == 2018
filtered = merged[mask]
filtered_group = filtered.groupby(by=['genres'])['rating'].agg(['mean', 'count'])
mask2 = filtered_group['count'] > 10
result = filtered_group[mask2].sort_values(by = ['mean'], ascending=[False])
print(result.head(1))

                             mean  count
genres                                  
Action|Adventure|Sci-Fi  3.928571     14


Задание 8.7
Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:

In [145]:
merged['year_rating'] = merged['date'].dt.year

display(merged)

,userId,movieId,rating,date,title,genres,year_release,year_rating
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,2000
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995,2000
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995,2000
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995,2000
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995,2000
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017,2017
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017,2017
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017,2017
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017,2017


In [154]:
pivot = merged.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    aggfunc='mean',
    fill_value=0
)
display(round(pivot, 2))

year_rating,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
genres,,,,,,,,,,,,,,,,,,,,,
(no genres listed),0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,3.50,3.00,3.68
Action,2.73,3.54,0.0,0.0,2.59,3.00,2.75,3.83,2.70,3.36,...,0.00,3.50,4.50,2.62,2.50,0.00,3.10,3.55,2.64,2.59
Action|Adventure,3.45,4.15,4.2,4.0,3.74,3.50,4.30,3.28,4.14,3.41,...,3.71,3.44,3.89,3.90,3.83,3.88,3.56,3.72,3.53,3.40
Action|Adventure|Animation,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,3.00,3.80,3.75,3.38,3.65
Action|Adventure|Animation|Children,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,4.00,3.00,3.50,3.67,3.83,3.50,3.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sci-Fi|Thriller,2.67,3.40,0.0,4.0,2.14,2.50,3.60,3.14,0.00,2.75,...,4.00,2.50,1.83,3.50,3.83,0.00,3.76,3.92,3.75,3.56
Sci-Fi|Thriller|IMAX,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,4.00,3.67,4.00,4.00,4.00,3.67,2.50,0.00
Thriller,3.84,3.92,3.8,3.7,3.09,3.48,3.58,3.25,3.46,3.41,...,2.96,2.83,3.62,3.08,3.67,4.00,2.91,3.43,3.15,3.30


A.
За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.

In [225]:
col = pivot.columns
a = []
b = True
i=0
for c in col:
    a.append(pivot[c]['Action|Adventure'] > 3)
    b = b * a[i]
    i=i+1
display(b)

True

B.
Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.

In [276]:
filtered = merged.loc[merged['genres'] == 'Action|Adventure|Animation|Children|Comedy|IMAX', ['rating','year_rating']]
result = filtered.sort_values(by = 'rating', ascending=[False])
display(result.head())

,rating,year_rating
90181,5.0,2018
90159,5.0,2018
36159,5.0,2012
60283,5.0,2013
36127,4.5,2012


In [262]:
pivot.loc['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values(ascending=False).head()

year_rating
2013    5.000000
2018    3.700000
2016    3.666667
2012    3.550000
2014    3.500000
Name: Action|Adventure|Animation|Children|Comedy|IMAX, dtype: float64

C.
Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.

In [316]:
pivot.loc['Animation|Children|Mystery'][2018]

5.0

In [319]:
pivot.loc[:][2018].sort_values(ascending=False).head(10)

genres
Comedy|Horror|Mystery                        5.0
Comedy|Fantasy|Horror|Musical|Thriller       5.0
Animation|Children|Mystery                   5.0
Comedy|Crime|Horror|Thriller                 5.0
Drama|Horror|Mystery|Sci-Fi|Thriller         5.0
Adventure|Children|Comedy|Fantasy|Mystery    5.0
Action|Crime|Thriller|Western                5.0
Comedy|Crime|Fantasy                         5.0
Crime|Mystery                                5.0
Drama|Romance|War                            5.0
Name: 2018, dtype: float64

D.
Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).

In [317]:
pivot.loc['Comedy']

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64